In [1]:
from IPython import display

In [2]:
try:
    import nbformat
except:
    !pip install nbformat
display.clear_output()

In [3]:
%run /content/drive/MyDrive/house_predict/process/test_27102025/model/train.ipynb

+ action: train_feat01
+ feat_path: /content/drive/MyDrive/house_predict/process/exps/data.npz
+ seed: 42
+ exp_dir: /content/drive/MyDrive/house_predict/process/exps
+ exp_name: trainbase_27102025
+ data_dir: /content/drive/MyDrive/house_predict/data
+ verbose: True
+ save_dir: /content/drive/MyDrive/house_predict/process/exps/trainbase_27102025
---------- information ----------
train-col: {'BsmtFinType2', 'BsmtExposure', 'PoolArea', 'GarageFinish', 'Functional', 'LotFrontage', 'OpenPorchSF', 'HeatingQC', 'MoSold', 'LotConfig', 'SaleType', 'BldgType', 'Fireplaces', 'ExterCond', 'MiscFeature', 'Street', 'YrSold', 'FullBath', 'Electrical', 'HalfBath', 'YearBuilt', 'BsmtUnfSF', 'EnclosedPorch', 'OverallQual', 'MSSubClass', 'OverallCond', 'BsmtFinType1', 'BsmtHalfBath', 'TotalBsmtSF', 'BsmtQual', 'RoofMatl', 'GarageYrBlt', 'GarageQual', 'Fence', 'YearRemodAdd', 'Id', 'LotShape', 'Condition2', 'KitchenAbvGr', 'Heating', 'PoolQC', 'FireplaceQu', 'LandSlope', 'MSZoning', 'GarageArea', 'Scree

In [4]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import os

data = np.load("/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/data01.npz", allow_pickle=True)

In [5]:
with np.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/data01.npz') as z:
    print(z.files)

['train_data', 'test_data', 'train_columns', 'test_columns', 'allow_pickle']


In [6]:
# Tạo DataFrame từ file nén
df_train = pd.DataFrame(data['train_data'], columns=data['train_columns'])
df_test = pd.DataFrame(data['test_data'], columns=data['test_columns'])

# Tách nhãn ra trước khi encode
y_train = df_train['SalePrice']
df_train = df_train.drop(columns=['SalePrice'])

# Mã hoá one-hot cho cả train và test
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

# Căn chỉnh lại 2 bộ dữ liệu (đảm bảo cùng số cột)
df_train, df_test = df_train.align(df_test, join='inner', axis=1)

# Gán dữ liệu huấn luyện & kiểm tra
X_train = df_train
X_test = df_test


# LinearRegression


In [7]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Tạo pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())  # hoặc Lasso(), bạn có thể thay thử
])

# Thiết lập hyperparameters
param_grid = {
    'regressor': [Ridge(), Lasso()],
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

# Sử dụng K-Fold cho regression (không Stratified)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search với nhiều scoring metrics
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring={'r2': 'r2', 'rmse': 'neg_root_mean_squared_error'},
    refit='r2',  # Chọn model có R² cao nhất
    n_jobs=-1,
    verbose=1
)

# Train model
print("\n Bắt đầu training với GridSearch...")
grid_search.fit(X_train, y_train)

# Kết quả
print("\n Best parameters:", grid_search.best_params_)
print("🏆 Best cross-validation R²: {:.4f}".format(grid_search.best_score_))

# Dự đoán trên train
y_pred_train = grid_search.predict(X_train)
rmse = mean_squared_error(y_train, y_pred_train) ** 0.5
r2 = r2_score(y_train, y_pred_train)
print(f"\n📊 Train RMSE: {rmse:.4f}")
print(f"📈 Train R²: {r2:.4f}")

# Hiển thị tất cả kết quả
cv_results = grid_search.cv_results_
print("\n Cross-validation results:")
for i in range(len(cv_results['params'])):
    print(f"Params: {cv_results['params'][i]}")
    print(f"  R²: {cv_results['mean_test_r2'][i]:.4f} (+/- {cv_results['std_test_r2'][i]:.4f})")
    print(f"  RMSE: {-cv_results['mean_test_rmse'][i]:.4f} (+/- {cv_results['std_test_rmse'][i]:.4f})")



 Bắt đầu training với GridSearch...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

 Best parameters: {'regressor': Lasso(), 'regressor__alpha': 100}
🏆 Best cross-validation R²: 0.7688

📊 Train RMSE: 4651.8453
📈 Train R²: 0.9966

 Cross-validation results:
Params: {'regressor': Ridge(), 'regressor__alpha': 0.001}
  R²: 0.7430 (+/- 0.0653)
  RMSE: 39767.2980 (+/- 4690.3020)
Params: {'regressor': Ridge(), 'regressor__alpha': 0.01}
  R²: 0.7430 (+/- 0.0653)
  RMSE: 39766.4530 (+/- 4689.9223)
Params: {'regressor': Ridge(), 'regressor__alpha': 0.1}
  R²: 0.7431 (+/- 0.0653)
  RMSE: 39758.2043 (+/- 4686.2279)
Params: {'regressor': Ridge(), 'regressor__alpha': 1}
  R²: 0.7439 (+/- 0.0649)
  RMSE: 39691.7124 (+/- 4657.4613)
Params: {'regressor': Ridge(), 'regressor__alpha': 10}
  R²: 0.7471 (+/- 0.0640)
  RMSE: 39445.9436 (+/- 4584.8878)
Params: {'regressor': Ridge(), 'regressor__alpha': 100}
  R²: 0.7595 (+/- 0.0600)
  RMSE: 38485.3862 (+/- 4510.3556)
Params: {'regressor': Lass

In [8]:
# Model tốt nhất
best_model_lr = grid_search.best_estimator_

# Dự đoán trên tập train
y_pred_train = best_model_lr.predict(X_train)

# Đánh giá mô hình
train_r2 = r2_score(y_train, y_pred_train)
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)
train_rmse = train_mse ** 0.5

print("\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"R² Score      : {train_r2:.4f}")
print(f"MAE           : {train_mae:.4f}")
print(f"MSE           : {train_mse:.4f}")
print(f"RMSE          : {train_rmse:.4f}")


=== KẾT QUẢ TRÊN TRAINING SET ===
R² Score      : 0.9966
MAE           : 3627.9493
MSE           : 21639665.0946
RMSE          : 4651.8453


In [10]:
import joblib

# Giả sử model đã train xong
# ví dụ: best_model_lr.fit(X_train, y_train)

# Lưu riêng model
joblib.dump(best_model_lr, '/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/linear_regression_model.pkl')
print("✅ Model đã được lưu vào linear_regression_model.pkl")

# Lưu thêm thông tin đi kèm (đề phòng sau này dùng lại)
package = {
    'model': best_model_lr,
    'scaler': scaler if 'scaler' in locals() else None,
    'feature_names': X_train.columns.tolist(),
    'training_date': '2025-10-27'
}

joblib.dump(package, '/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/linear_regression_package.pkl')
print("✅ Package (model + scaler + features) đã được lưu")


✅ Model đã được lưu vào linear_regression_model.pkl
✅ Package (model + scaler + features) đã được lưu


In [11]:
# 🔹 Load model đã lưu
model_package = joblib.load('/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/linear_regression_package.pkl')
model = model_package['model']

# 🔹 Nếu có scaler thì load ra
scaler = model_package.get('scaler', None)

# 🔹 Đọc lại file test gốc để lấy cột Id
df_test_raw = pd.read_csv('/content/drive/MyDrive/house_predict/data/test.csv')
ids = df_test_raw['Id']  # Lưu cột Id gốc (từ 1461–2919)

# 🔹 Scale dữ liệu nếu có scaler
if scaler is not None:
    X_test = scaler.transform(df_test)
else:
    X_test = df_test.values

# 🔹 Dự đoán
y_pred_test = model.predict(X_test)

# 🔹 Tạo file submission
submission = pd.DataFrame({
    'Id': ids,
    'SalePrice': y_pred_test
})

# 🔹 Lưu file CSV
submission.to_csv('/content/drive/MyDrive/house_predict/process/exps/trainbase_27102025/submission.csv', index=False)
print("💾 File submission.csv đã được tạo thành công!")

💾 File submission.csv đã được tạo thành công!
